In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams["figure.figsize"] = (12, 8)
pd.set_option('display.max_columns', None)

sns.set_style("darkgrid")
%matplotlib inline

In [2]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from file_handler import FileHandler
from df_overview import DfOverview
from df_cleaner import DfCleaner
from df_outlier import DfOutlier
from plots import *

2022-05-24 05:20:01,694 — DfOverview — DEBUG — Loaded successfully!
2022-05-24 05:20:01,698 — DfCleaner — DEBUG — Loaded successfully!
2022-05-24 05:20:01,703 — DfOutlier — DEBUG — Loaded successfully!


#### Read the Datasets

In [3]:
file_handler = FileHandler()
df_train = file_handler.read_csv("../data/train.csv")
df_test = file_handler.read_csv("../data/test.csv")
df_store = file_handler.read_csv("../data/store.csv")

2022-05-24 05:20:03,565 — FileHandler — DEBUG — file read as csv
2022-05-24 05:20:03,600 — FileHandler — DEBUG — file read as csv
2022-05-24 05:20:03,607 — FileHandler — DEBUG — file read as csv


c:\Users\Sebli\Desktop\10x Files\Week 3\Pharmaceutical-Sales-Prediction\scripts\file_handler.py:28: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
